# EDA & Cleaning

In [1]:
#IMPORTS
import warnings; warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Markdown, display
pd.options.display.float_format = '{:.3f}'.format
plt.rcParams['figure.figsize'] = (10, 2)

In [2]:
#LOAD LOANSTATS
directory = '../../data/'
ls = pd.read_hdf(directory + 'Ls_MERGED.h5', 'ls_MERGED') # HDF5
ls.sort_index(axis=1, inplace=True)

In [3]:
#LOAD DATA DICTIONARY
sheet_dict = pd.read_excel(directory + 'LCDataDictionary.xlsx', sheet_name=None)
data_dict = {}
for key in sheet_dict:
    for index, row in sheet_dict[key].iterrows():
        if type(row[0]) != float:
            data_dict[row[0].strip()] = row[1]

## 1. Term-Complete Representative Subset

'Term-complete' loans have completed their full term whereas 'in-force' loans have not completed their term.  To get the most representative outcome information we first remove the loan instances that are not term-complete. Also, LC no longer offers loans of the lowest grades F or G, so we removed these loans from the dataset.

In [4]:
#TERM-COMPLETE GRADES A-E
completed_36 = (ls['issue_d'] < '2015-04-01') & (ls['term']  == ' 36 months')
completed_60 = (ls['issue_d'] < '2013-04-01') & (ls['term']  == ' 60 months')
ls = ls[completed_36 | completed_60]
ls = ls[(ls['grade'] != 'F') & (ls['grade'] != 'G')]

## 2. Inconsequential Variable Removal

We removed variables from the `loanstats` dataset that would not be meaningful in the modeling for the following reasons:
1. LC recently began reporting some new variables which are not reported in the term-complete subset. 
2. LC recently began accepting joint loans and variables related to coborrowers are empty in the term-complete subset. 
3. We designed three outcome features based on the set of dependent variables (see [Outcome Feature Design](https://cs109group67.github.io/lendingclub/EDA.html#3-outcome-feature-design))
4. Some variables are non-standard text descriptions or otherwise inconsequential

In [5]:
#1 NEW
empty = ['all_util', 'il_util', 'inq_fi', 'inq_last_12m', 'max_bal_bc', 
         'mths_since_rcnt_il', 'open_acc_6m', 'open_act_il', 'open_il_12m', 
         'open_il_24m', 'open_rv_12m', 'open_rv_24m','total_bal_il', 'total_cu_tl']

#2 CO-BORROWER
joint = ['application_type', 'annual_inc_joint', 'dti_joint', 'revol_bal_joint', 
         'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 
         'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths', 'sec_app_mort_acc', 
         'sec_app_mths_since_last_major_derog', 'sec_app_num_rev_accts', 'sec_app_open_acc', 
         'sec_app_open_act_il', 'sec_app_revol_util', 'verification_status_joint']

#3 DEPENDENT
dependent = [# Payment Variables (11): 
              'issue_d', 'last_pymnt_amnt', 'last_pymnt_d', 'loan_status', 
              'next_pymnt_d', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 
              'total_pymnt_inv', 'total_rec_int', 'total_rec_prncp', 
             # Hardship/Collections/Settlements (27)
              'collection_recovery_fee', 'debt_settlement_flag', 'debt_settlement_flag_date', 
              'deferral_term', 'hardship_amount', 'hardship_dpd', 'hardship_end_date', 
              'hardship_flag', 'hardship_last_payment_amount','hardship_length', 'hardship_loan_status', 
              'hardship_payoff_balance_amount', 'hardship_reason', 'hardship_start_date', 
              'hardship_status', 'hardship_type', 'last_credit_pull_d', 
              'orig_projected_additional_accrued_interest', 'payment_plan_start_date', 'pymnt_plan', 
              'recoveries', 'settlement_amount', 'settlement_date', 'settlement_percentage', 
              'settlement_status', 'settlement_term', 'total_rec_late_fee']

#4 OTHER INCONSEQUENTIAL
inconsequential = ['addr_state', # not useful as dummy variable
                   'dataset', # just indicates the dataset
                   'desc', # non-standard text description
                   'disbursement_method', # just indicates cash or direct_pay
                   'emp_title', # non-standard text description
                   'funded_amnt', # redundant with loan_amount
                   'funded_amnt_inv', # redundant with loan_amount
                   'grade', # redundant when using sub_grade
                   'initial_list_status', # possible values are w or f
                   'title'] # non-standard text description
#                   'zip_code'] # we could make into dummies, but there are 954 of them

In [6]:
ls.drop(empty, axis=1, inplace=True)
ls.drop(joint, axis=1, inplace=True)
ls.drop(inconsequential, axis=1, inplace=True)

## 3. Outcome Feature Design

We designed 3 outcome features to represent how well or poorly the loan performed: `OUT_Class`, `OUT_Principle_Repaid_Percentage` and `OUT_Monthly_Rate_of_Return`.

### 3A. `OUT_Class`

This outcome variable is a binary classification of whether the loan has been Fully Repaid (1) or Not Fully Repaid (0). The percentage of loans that have been fully repaid is 85.9%.

In [7]:
ls['OUT_Class'] = 0
ls.loc[ls['loan_status'].str.contains('Fully Paid'), 'OUT_Class'] = 1

In [8]:
ls.loc[ls['loan_status'].str.contains('Current'), 'OUT_Class'] = 1

### 3B. `OUT_Principle_Repaid_Percentage`

This outcome variable represents the percentage of loan principle that has been repaid. The average principal repaid percentage is 91.5%.

In [9]:
ls['OUT_Principle_Repaid_Percentage'] = ls['total_rec_prncp'] / ls['loan_amnt']

### 3C. `OUT_Monthly_Rate_Of_Return`

This outcome variable represents the simple monthly rate of return that investors recieved by holding the loan. This is the most comprehensive of the three outcome features because it takes into account the total amount repaid (including interest) for the effective term of the loan. The median monthly rate of return is 0.6%.

In [10]:
#NET_REPAYMENT: amount repaid on the loan net of the loan amount
Net_Repayment = ls['total_pymnt'] - ls['loan_amnt']

#REPAYMENT_PERIOD: amount of time it took to repay the loan or charge off
Repayment_Period = (ls['last_pymnt_d'].dt.to_period('M') - 
                    ls['issue_d'].dt.to_period('M')).replace([pd.NaT,0], 1)

#MONTHLY_RATE_OF_RETURN: simple monthly return accrued over the term of the loan
ls['OUT_Monthly_Rate_Of_Return'] = (Net_Repayment / Repayment_Period) / ls['loan_amnt']

## 4. Independent Feature Cleaning

We cleaned the independent variables with type conversions, dummy creation and outlier identification. Loans that had missing values in more than half of the independent variables were dropped while the remaining missing values were imputed with mean imputation or zero imputation. This section presents the custom functions that we used and the following section [EDA](https://cs109group67.github.io/lendingclub/EDA.html#5-eda) summarizes the cleaned features.

### 4A. Type Conversions

In [11]:
#EARLIEST_CR_LINE
ls['earliest_cr_line'] =  np.array((ls['issue_d'] - 
                                    ls['earliest_cr_line']).dt.days).reshape(-1,1)

#EMP_LENGTH
mapping = {'1 year': 1, '10+ years': 10, '2 years': 2, '3 years': 3, 
           '4 years': 4, '5 years': 5, '6 years': 6, '7 years': 7, 
           '8 years': 8, '9 years': 9, '< 1 year': 0}
ls['emp_length'] = ls['emp_length'].map(mapping)

#HOME_OWNERSHIP
ls['home_ownership'].replace({'ANY':'OTHER', 'NONE':'OTHER'}, inplace=True)

#INT_RATE
ls['int_rate'] = ls['int_rate'].str[:-1].astype(np.float)

#REVOL_UTIL
ls['revol_util'] = ls['revol_util'].str[:-1].astype(np.float)

#SUB_GRADE
mapping = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4}
ls['sub_grade'] = (ls['sub_grade'].apply(lambda x: x[0]).map(mapping).astype(int)*5 +
                         ls['sub_grade'].apply(lambda x: x[1]).astype(int)).astype(float)

#'MTHS_SINCE...' reciprocals
#use reciprocal so that 0='never occured' and 1/n+1='n month since'
mnths_since = ls.columns[ls.columns.str.contains('mo_sin|mths_since')]
for var in mnths_since:
    ls[var+'_R'] = 1 / (ls[var] + 1)
ls.drop(mnths_since, axis=1, inplace=True)

### 4B. Dummy Creation

In [12]:
#FUNCTION FOR DUMMY CREATION
def dummy_attr(attr):
    """ Create dummmies and drop original attribute"""
    global ls
    if attr not in list(ls): return
    prefix = 'D_' + attr
    dummies = pd.get_dummies(ls[attr], prefix=prefix)
    ls.drop([attr], axis=1, inplace=True)
    ls = pd.concat([ls, dummies], axis=1)

### 4C. Outlier Identification

In [13]:
#FUNCTION FOR OUTLIER DETECTION
ls['outlier'] = 0 # this column is incremented for identified outlier instances
def outlier_attr(attr, threshold):
    """ Identify outliers above threshold""" 
    outliers = ls[attr] > threshold
    ls['outlier'] = ls['outlier'] + outliers
    return outliers

### 4D. Missing Value Imputation

In [14]:
#REMOVE LOANS WITH MANY MISSING FEATURES
ls = ls[ls.isnull().sum(axis=1) < 30]

#FUNCTION FOR MISSING VALUE IMPUTATION
from sklearn.impute import SimpleImputer
def impute_attr(attr, strategy='median'):
    """ Impute missing values (via mean imputation or constant imputation)"""
    mnths_since = ls.columns[ls.columns.str.contains('mo_sin|mths_since')]
    if attr in mnths_since:
        imp = SimpleImputer(strategy='constant', fill_value=0)
    elif ls[attr].min() == 0:                                
        imp = SimpleImputer(strategy='constant', fill_value=0)
    else:
        imp = SimpleImputer(strategy='mean')
    ls[attr] = imp.fit_transform(ls[[attr]])

In [15]:
ls.drop(dependent, axis=1, inplace=True)
ls.drop('outlier', axis=1, inplace=True)

In [16]:
############ EXPORTING ##############
ls.to_hdf(directory + 'ls_CLEAN_withzip.h5', 'ls_CLEAN_withzip')